In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available ✅")
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available ❌")

In [ ]:
import pandas as pd
import numpy as np
import tqdm

In [ ]:
data = pd.read_csv("/content/train.csv")
data.head()


In [ ]:
seed = 42
data = data.sample(frac=0.35, random_state=seed)
data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    data.drop(columns=['price']),  # features
    data['price'],                 # target
    test_size=0.2,
    random_state=42
)

print(f"x_train, y_train: {(x_train.shape, y_train.shape)}")


In [ ]:
x_train.head()

In [ ]:
data['image_link']

In [ ]:
import torch
from torch import nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pandas as pd

In [ ]:
# Image preprocessing (as expected by ResNet)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
# Load pretrained ResNet50
model = models.resnet50(pretrained=True)
model = nn.Sequential(*list(model.children())[:-1])  # remove classifier
model.eval()

In [ ]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
def get_image_embedding(urls):
    """Handles single or multiple URLs per product (list or str)"""
    if isinstance(urls, str):
        urls = [urls]
    embeddings = []
    for url in urls:
        try:
            response = requests.get(url, timeout=10)
            img = Image.open(BytesIO(response.content)).convert('RGB')
            img_t = transform(img).unsqueeze(0).to(device)
            with torch.no_grad():
                emb = model(img_t).squeeze().cpu()
            embeddings.append(emb)
        except Exception as e:
            print(f"Error loading {url}: {e}")
    if embeddings:
        return torch.stack(embeddings).mean(dim=0)  # average for multiple imgs
    else:
        return torch.zeros(2048)  # if failed

In [ ]:
# Wrap with tqdm for progress bar
x_train['image_embedding'] = [
    get_image_embedding(url) for url in tqdm.tqdm(x_train['image_link'], desc="Processing Images")
]

print(x_train['image_embedding'].head())

In [ ]:
x_train.iloc[0,3]

In [ ]:
x_train.head()

In [ ]:
x_train['catalog_content'].head()

In [ ]:
for item in x_train['catalog_content']:
  print(item)

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import re

In [ ]:
def preprocess_catalog_text(text):
    if not isinstance(text, str):
        return ""

    # Extract specific fields (use regex with optional presence)
    item_name = re.search(r"Item Name:\s*(.*)", text)
    desc = re.search(r"Product Description:\s*(.*)", text, re.DOTALL)
    value = re.search(r"Value:\s*([\d.]+)", text)
    unit = re.search(r"Unit:\s*([A-Za-z]+)", text)

    # Extract all bullet points (1 to 5)
    bullet_points = re.findall(r"Bullet Point \d+:\s*(.*)", text)

    # Combine clean text parts
    text_parts = []
    if item_name: text_parts.append(item_name.group(1).strip())
    if bullet_points: text_parts.extend([bp.strip() for bp in bullet_points])
    if desc: text_parts.append(desc.group(1).strip())
    if value and unit:
        text_parts.append(f"Quantity: {value.group(1)} {unit.group(1)}")
    elif value:
        text_parts.append(f"Quantity: {value.group(1)}")

    return " ".join(text_parts)

In [ ]:
x_train["clean_text"] = x_train["catalog_content"].apply(preprocess_catalog_text)

In [ ]:
x_train["clean_text"][0:10]

In [ ]:
model_text = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

x_train["text_embedding"] = x_train["clean_text"].apply(lambda x: model_text.encode(x))
print(x_train["text_embedding"].iloc[0].shape)  # → (384,)


In [ ]:
x_train.head()

In [ ]:
import torch

def concat_embeddings(img_emb, txt_emb):
    img_emb = torch.tensor(img_emb, dtype=torch.float32)
    txt_emb = torch.tensor(txt_emb, dtype=torch.float32)
    return torch.cat([img_emb, txt_emb], dim=0)

x_train["combined_embedding"] = x_train.apply(
    lambda row: concat_embeddings(row["image_embedding"], row["text_embedding"]),
    axis=1
)


In [ ]:
x_train.head()